In [1]:
!pip install sentence-transformers

In [78]:
# from owlready2 import *
# onto = get_ontology("D:\Thesis\Ontology file\Wind energy\saref4ener.rdf").load()

# for cls in onto.classes():
#     if cls.namespace != onto.base_iri:
#         print(cls.name, cls.namespace)


In [79]:
# from rdflib import Graph, Namespace

# # Load the ontology
# g = Graph()
# g.parse("D:\Thesis\Ontology file\Wind energy\saref4ener.rdf")

# # Define the namespace of the ontology you want to remove classes from
# other_ontology_ns = Namespace("http://www.w3.org/2006/time#")

# # Use a SPARQL query to find all the classes from the other ontology
# query = """
# PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX owl: <http://www.w3.org/2002/07/owl#>

# SELECT ?class
# WHERE {
#     ?class rdf:type owl:Class .
#     FILTER (strstarts(str(?class), str(%s)) && (str(?class) = str(%s) || (str(?class) = str(%sDurationDescription) || str(?class) = str(%sTemporalEntity) )
# }
# """ % (other_ontology_ns, other_ontology_ns, other_ontology_ns)

# results = g.query(query)

# # Iterate over the results and remove each class
# for class_uri, in results:
#     g.remove((class_uri, None, None))

# # Save the modified ontology
# g.serialize("ontology_modified.owl", format='xml')


In [1]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Reading the sentences file
sentences_df = pd.read_csv("D:\Thesis\Ontology file\Wind energy\clean-ontology-file.csv")
sentences = sentences_df["comment"].tolist()

# cleaning the file - ontlogy_data

sentences_df = sentences_df.dropna(subset=['comment'])
sentences_df = sentences_df.loc[sentences_df['comment'].apply(lambda x: type(x)==str)]

sentences_df["comment"] = sentences_df["comment"].astype(str)

sentences = [x for x in sentences if type(x) != float]

# Reading the comments file
comments_df = pd.read_csv("D:\Thesis\Ontology file\Wind energy\clean-pdf-file.csv")
comments = comments_df["text_column"].tolist()

# cleaning the file - pdf file

comments_df = comments_df.dropna(subset=['text_column'])
comments_df = comments_df.loc[comments_df['text_column'].apply(lambda x: type(x)==str)]

comments_df["text_column"] = comments_df["text_column"].astype(str)

comments = [x for x in comments if type(x) != float]

# Encoding the sentences and comments
sentences_embeddings = sbert_model.encode(sentences)
comments_embeddings = sbert_model.encode(comments)

# Creating the final list for storing the cosine similarity values
final_list = []

# Iterating through the sentences and comments
for original_sentence, embedded_sentence, original_comment, embedded_comment in zip(sentences, sentences_embeddings, comments, comments_embeddings):
    # Calculating the cosine similarity
    sim = cosine_similarity([embedded_sentence], [embedded_comment])[0][0]
    # Storing the cosine similarity values in the final list
    final_list.append([original_sentence, original_comment, sim, sentences_df["label"].tolist()[sentences.index(original_sentence)], hash(original_sentence), hash(original_comment)])

# Create a DataFrame
df = pd.DataFrame(final_list, columns=["Sentence","Comment", "Similarity","Class_label","Sentence_hash","Comment_hash"])


# Print the DataFrame
print(df)

                                             Sentence  \
0   a collection of power sequences for a certain ...   
1   if state is set to running or paused and the s...   
2   in the load control it expresses the possible ...   
3   a possible type of power in a slot that repres...   
4   the duration description for the slot time dur...   
..                                                ...   
58  a way to model curves of power and energy over...   
59  the active maximum duration a power sequence c...   
60  the duration description for the active durati...   
61  the duration of a slot shall be present in cas...   
62  the current state of the power sequence it can...   

                                              Comment  Similarity  \
0     Gyancity Journal of Engineering and Technology     0.620684   
1                          Vol4 No2 pp2937 July 2018     0.382506   
2              ISSN24560065 DOI 1021058gjet201842004     0.388732   
3                                      

In [3]:
# import pandas as pd

# df = pd.read_csv('D:\Thesis\Ontology file\Wind energy\Final-output1.csv')
# df

In [7]:
import pandas as pd
from owlready2 import *

# Loading the ontology file
onto = get_ontology("D:/Thesis/Ontology file/Wind energy/new ontology file.owl").load()
#onto = get_ontology("D:\Thesis\Ontology file\Wind energy\saref.rdf").load()

# # Extract all classes
classes = list(onto.classes())

# Create a list of classes
class_list = []
for cls in classes:
    class_list.append(cls.name)

# Create a DataFrame
df1 = pd.DataFrame(class_list, columns=["Classes"])

# Print the DataFrame
print(df1)

                               Classes
0                               Energy
1                      ActivationDelay
2   ActivationDelayDurationDescription
3            ActiveDurationDescription
4                    ActiveDurationMax
..                                 ...
58              PowerStandardDeviation
59                  RemainingPauseTime
60         SlotTimeDurationDescription
61        StartTimeDurationDescription
62       TimePeriodDurationDescription

[63 rows x 1 columns]


In [17]:
import pandas as pd
import csv
from owlready2 import *

# load ontology
onto = get_ontology("D:/Thesis/Ontology file/Wind energy/new ontology file.owl").load()

# combine two dataframes
result = pd.concat([df, df1], axis=1, join='inner')
display(result)



# remove any rows with NaN values
result = result.dropna()


# # iterate through rows
# for index, row in result.iterrows():
#     class_name = row['Classes']
#     print(class_name)
#     target_class = onto[str(class_name)]
#     print(target_class)
#     sentence_hash = row['Sentence_hash']
#     print(sentence_hash)
#     if target_class is not None:
#         instance_of_class = target_class(sentence_hash)
#         # Add other properties 
#         #print(instance_of_class)

for index, row in result.iterrows():
    class_name = (result['Classes'][index])
    print(class_name)
    targetClass = onto[str(class_name)]
    print(targetClass)
    sentence_hash = (result['Sentence_hash'][index])
    print(sentence_hash)
    instanceOfClassDomain = targetClass(sentence_hash)
    print(instanceOfClassDomain)

    
#debugging steps to solve the errors
# import os

# # check file path
# file_path = "D:\Thesis\Ontology file\Wind energy\ontology1.owl"
# print("File path: ", os.path.abspath(file_path))

# # check if individuals are being added correctly
# print("Individuals in ontology: ", onto.individuals())

# # check for errors in data
# for index, row in result.iterrows():
#     class_name = (result['Classes'][index])
#     targetClass = onto[str(class_name)]
#     sentence_hash = (result['Sentence_hash'][index])
#     print("Class name: ", class_name)
#     print("Target class: ", targetClass)
#     print("Sentence hash: ", sentence_hash)

# check if code is properly iterating through dataframe
# print("Dataframe shape: ", result.shape)

# # check if individuals are being created correctly
# for index, row in result.iterrows():
#     class_name = (result['Classes'][index])
#     targetClass = onto[str(class_name)]
#     sentence_hash = (result['Sentence_hash'][index])
#     if targetClass is not None:
#         instanceOfClassDomain = targetClass(sentence_hash)
#         print("Individual: ", instanceOfClassDomain)

# onto.add_individual(instanceOfClassDomain)
# print("Individuals in ontology: ", onto.individuals())


# save ontology
onto.save("D:\Thesis\Ontology file\Wind energy\final-ontology1")


,Sentence,Comment,Similarity,Class_label,Sentence_hash,Comment_hash,Classes
0,a collection of power sequences for a certain ...,Gyancity Journal of Engineering and Technology,0.620684,alternatives group,-5431470730033479292,7881610776741779150,Energy
1,if state is set to running or paused and the s...,Vol4 No2 pp2937 July 2018,0.382506,elapsed slot time,-661056241173161469,8634859702327399456,ActivationDelay
2,in the load control it expresses the possible ...,ISSN24560065 DOI 1021058gjet201842004,0.388732,load control event state,-2080104192661936003,2666688400359404214,ActivationDelayDurationDescription
3,a possible type of power in a slot that repres...,30,0.396654,power expected,3591455369593246128,1115495798641711337,ActiveDurationDescription
4,the duration description for the slot time dur...,As per the report of The European Wind Energy...,0.326266,slot time duration description,8156588868403101559,544915238057651877,ActiveDurationMax
...,...,...,...,...,...,...,...
58,a way to model curves of power and energy over...,Wind turbines fabricate power to abuse the mod...,0.528715,power profile,2310004797765517011,7523815429035901227,PowerStandardDeviation
59,the active maximum duration a power sequence c...,The breeze,0.427835,active duration max,-1108193858188026637,-2666957549909753701,RemainingPauseTime
60,the duration description for the active durati...,could be a clean and property supply of fuel i...,0.187786,active duration description,-3332844801844070359,-7522724689225723525,SlotTimeDurationDescription
61,the duration of a slot shall be present in cas...,restored with the vitality of the sun,0.433783,default duration,-1318936976647841386,7877117301230176111,StartTimeDurationDescription


Energy
new ontology file.Energy
-5431470730033479292
new ontology file.-5431470730033479292
ActivationDelay
new ontology file.ActivationDelay
-661056241173161469
new ontology file.-661056241173161469
ActivationDelayDurationDescription
new ontology file.ActivationDelayDurationDescription
-2080104192661936003
new ontology file.-2080104192661936003
ActiveDurationDescription
new ontology file.ActiveDurationDescription
3591455369593246128
new ontology file.3591455369593246128
ActiveDurationMax
new ontology file.ActiveDurationMax
8156588868403101559
new ontology file.8156588868403101559
ActiveDurationMin
new ontology file.ActiveDurationMin
2219623917791085710
new ontology file.2219623917791085710
ActiveDurationSumMax
new ontology file.ActiveDurationSumMax
6280012323938255290
new ontology file.6280012323938255290
ActiveDurationSumMin
new ontology file.ActiveDurationSumMin
6745664789021616045
new ontology file.6745664789021616045
ActuatorLevel
new ontology file.ActuatorLevel
-80553820981373477

# Visualisation of the created ontology

In [14]:
pip install pyvis

Note: you may need to restart the kernel to use updated packages.
     -------------------------------------- 748.9/748.9 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 40.5/40.5 kB 1.9 MB/s eta 0:00:00
  Created wheel for pyvis: filename=pyvis-0.3.1-py3-none-any.whl size=755835 sha256=6942b03155003cf069545972eaf5a90968662454640bd9b7b05eaad7db304ad4
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\1f\3f\fd\aca0ca220a6cfa8e7258df8cf557f8680db0ff1044d897bbc1
Successfully built pyvis


In [ ]:
from owlready2 import *

onto = get_ontology("D:\Thesis\Ontology file\Wind energy\final-ontology1").load()

# Extract classes
classes = list(onto.classes())
print("Classes:")
for cls in classes:
    print(cls.name)

# Extract object properties
properties = list(onto.object_properties())
print("\nObject Properties:")
for prop in properties:
    print("{} - Domain: {}, Range: {}".format(prop.name, prop.domain, prop.range))

# Extract individuals
individuals = list(onto.individuals())
print("\nIndividuals:")
for ind in individuals:
    print(ind.name)

# Extract relationships
relationships = []
for prop in properties:
    for domain in prop.domain:
        for range in prop.range:
            relationships.append((domain.name, prop.name, range.name))
print("\nRelationships:")
for rel in relationships:
    print("{} is a {} of {}".format(*rel))


In [1]:
from pyvis.network import Network
from owlready2 import *
onto = get_ontology("D:\Thesis\Ontology file\Wind energy\ontology1").load()

# Create a new Network object
net = Network(notebook=True)

# Iterate through the ontology's classes and add them as nodes to the network
for cls in onto.classes():
    net.add_node(cls.name)

# Iterate through the ontology's object properties and add them as edges to the network
for prop in onto.object_properties():
    for domain in prop.domain:
        for range in prop.range:
            net.add_edge(domain.name, range.name, title=prop.name)

# Display the network
net.show("D:\Thesis\Ontology file\Wind energy\ontology_network.html")


Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
